In [1]:
import numpy as np
from scipy import stats
import math
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
import pandas as pd
import geopandas as gpd

In [13]:
#read in the data
q1 = gpd.read_file('../data/processed-labels/clusters/clusters_q1.shp')
q2 = gpd.read_file('../data/processed-labels/clusters/clusters_q2.shp')
q3 = gpd.read_file('../data/processed-labels/clusters/clusters_q3.shp')
q4 = gpd.read_file('../data/processed-labels/clusters/clusters_q4.shp')

In [14]:
q2

,label_id,label_type,severity,gsv_panora,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,cluster_id,clustered,geometry
0,86461,CurbRamp,1.0,ZTCzCkmt9uG-TFqJEttrtg,2,323.580353,-27.50000,272.483002,6.449974,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.697720,-122.320244,196,1,POINT (-122.32024 47.69772)
1,86624,CurbRamp,1.0,Eo-oKwHk5rBFeIHsJa_OQQ,3,311.968750,-10.84375,90.119415,0.630173,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.712048,-122.290810,2358,0,POINT (-122.29081 47.71205)
2,86767,CurbRamp,1.0,Z1u8iq5TEuS8-n_kZKRc8A,1,174.437500,-5.87500,156.979935,-0.590858,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.700146,-122.283333,6961,0,POINT (-122.28333 47.70015)
3,86768,CurbRamp,1.0,Z1u8iq5TEuS8-n_kZKRc8A,1,128.500000,-13.00000,156.979935,-0.590858,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.700150,-122.283104,295,0,POINT (-122.28310 47.70015)
4,86808,CurbRamp,1.0,WfExyzpW559Rn4aEeuYMng,1,98.000000,-16.43750,282.873047,4.660347,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.693405,-122.276970,2734,0,POINT (-122.27697 47.69341)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30301,231609,Signal,NaN,jVLp7QRXH9TyDjbUXpXTTw,1,358.750000,-8.12500,289.983826,-0.280980,c11b44ac-3fa8-45f8-acf0-de27da3e1428,47.694157,-122.320427,9,0,POINT (-122.32043 47.69416)
30302,210508,Signal,NaN,GQSjiaAqrHylEWnSH_auzw,1,43.125000,-22.43750,171.160873,0.807304,4cac48cc-452d-40c1-8f16-9f54d6da1a6d,47.692154,-122.317436,4,0,POINT (-122.31744 47.69215)
30303,231604,Signal,NaN,jVLp7QRXH9TyDjbUXpXTTw,1,358.000000,-8.12500,289.983826,-0.280980,c11b44ac-3fa8-45f8-acf0-de27da3e1428,47.694210,-122.320419,10,0,POINT (-122.32042 47.69421)
30304,231605,Signal,NaN,jVLp7QRXH9TyDjbUXpXTTw,1,358.000000,-8.12500,289.983826,-0.280980,c11b44ac-3fa8-45f8-acf0-de27da3e1428,47.694252,-122.320320,2,0,POINT (-122.32032 47.69425)


In [15]:
q1['count'] = q1.groupby('cluster_id')['cluster_id'].transform('count')
q2['count'] = q2.groupby('cluster_id')['cluster_id'].transform('count')
q3['count'] = q3.groupby('cluster_id')['cluster_id'].transform('count')
q4['count'] = q4.groupby('cluster_id')['cluster_id'].transform('count')

In [16]:
q2['clustered'].value_counts()

0    19573
1    10733
Name: clustered, dtype: int64

In [17]:
q1['clustered'].value_counts()

0    34073
1    21728
Name: clustered, dtype: int64

In [18]:
q3['clustered'].value_counts()

0    36860
1    23521
Name: clustered, dtype: int64

In [19]:
q4['clustered'].value_counts()

0    32358
1    18937
Name: clustered, dtype: int64

In [20]:
#select only label_id label_type, clustered and geometry
q1 = q1[['label_id', 'clustered', 'count']]
q2 = q2[['label_id', 'clustered','count']]
q3 = q3[['label_id', 'clustered','count']]
q4 = q4[['label_id', 'clustered','count']]

In [21]:
# concatenate the dataframes q1 q2 q3 q4
df = pd.concat([q1, q2, q3, q4], ignore_index=True)

In [22]:
df['label_id'].nunique()

195501

In [23]:
# if label_id appears more than once keep clustered = 1
df = df.groupby('label_id').max()

In [24]:
#reset index
df = df.reset_index()

In [25]:
df

,label_id,clustered,count
0,9,1,7
1,10,1,6
2,11,0,9
3,12,0,2
4,13,0,8
...,...,...,...
195496,233233,0,5
195497,233234,0,6
195498,233235,0,5
195499,233236,1,7


In [30]:
labels = gpd.read_file('../data/processed-labels/seattle_labels_all/seattle_labels_all.shp')

In [31]:
#merge df with labels how inner
df1 = pd.merge(df, labels, on='label_id', how='outer')
#select only label_id and clustered
df1 = df1[['label_id', 'clustered','count']]
#fill na with 0 
df1 = df1.fillna(0)

In [32]:
df1

,label_id,clustered,count
0,9,1.0,7.0
1,10,1.0,6.0
2,11,0.0,9.0
3,12,0.0,2.0
4,13,0.0,8.0
...,...,...,...
195538,116833,0.0,0.0
195539,37995,0.0,0.0
195540,37983,0.0,0.0
195541,37989,0.0,0.0


In [34]:
#export to csv
df1.to_csv('../data/features/clustered.csv', index=False)